# XGBoost

- Vantagem: Conjuntos de Dados de Médio a Grande Porte - O XGBoost tende a funcionar melhor em conjuntos de dados grandes, onde pode lidar com eficiência com muitas características e amostras.
- Tem muitos parâmetros sensíveis que podem mudar drasticamente o modelo.

## Gradient Boosted Decision Tree

- É baseado na diferença do que a árvore anterior errou
- learning_rate = 0.1 (ex.)
- previsão = learning_rate * arv1 + learning_rate * arv2
- A ideia de treinar a partir dos erros anteriores: _Boost_
- Uma implementação do Boost, o que você tenta treinar se baseia na derivada do erro anterior: _Gradient_ <br> **Obs:** Existem outros tipos de Boost.

### Árvore 1 
```
ex1 - 100 -> 100 
ex2 - 120 -> 100
ex3 - 150 -> 130
ex4 - 80  -> 70
ex5 - 50  -> 60
```

### Árvore 2 

```
ex1 - (100-100) = 0   -> 10
ex2 - (120-100) = 20  -> 10
ex3 - (150-130) = 20  -> 20
ex4 - (80-70)   = 10  -> 0
ex5 - (50-60)   = -10 -> -5
```

### Árvore 3

```
ex1 - (10+100-100) = -10  -> 5 
ex2 - (10+100-120) = -10  -> -3
ex3 - (20+130-150) =  0   -> (...)
ex4 - (0+70-80)    = -10  -> 
ex5 - (-5+60-50)   = -5   -> 
```

y real = 100 <br>
arv1 = 100<br>
arv2 = 10<br>
previsão xgboost, gbdt -> 110<br>
ou seja, com o passar das árvores cada árvore vai tendo menos importância. 

## Colocando em prática

### Exemplo 1

Importando csv

In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv("./sample_data/california_housing_train.csv")
data.head()

Bibliotecas 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

Fazendo previsões

In [ ]:
X = data.drop("median_house_value", axis=1)
y = data['median_house_value']

Xtr, Xval, ytr, yval = train_test_split(X, y, test_size=0.5, random_state=0)

arveres_poderosas = XGBRegressor(learning_rate=0.08, 
                                 n_estimators=1300,
                                 max_depth=6,
                                 min_child_weight=1,
                                 subsample=0.75,
                                 colsample_bynode=0.5,
                                 random_state=0,
                                 #gpu_id='0',
                                 #tree_method='gpu_hist',
                                 #num_parallel_tree=3,
                                 booster="gbtree",
                                 objective='reg:squarederror'
                                 )
arveres_poderosas.fit(Xtr, ytr)

p = arveres_poderosas.predict(Xval)

np.sqrt(mean_squared_error(yval, p))

### Exemplo 2

Procedimento padrão

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer

data = pd.read_csv('../input/train.csv')
data.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = data.SalePrice
X = data.drop(['SalePrice'], axis=1).select_dtypes(exclude=['object'])
train_X, test_X, train_y, test_y = train_test_split(X.as_matrix(), y.as_matrix(), test_size=0.25)

my_imputer = Imputer()
train_X = my_imputer.fit_transform(train_X)
test_X = my_imputer.transform(test_X)

Ajustando modelo

In [ ]:
from xgboost import XGBRegressor

my_model = XGBRegressor()
# Add silent=True to avoid printing out updates with each cycle
my_model.fit(train_X, train_y, verbose=False)

Previsões e MAE

In [ ]:
# make predictions
predictions = my_model.predict(test_X)

from sklearn.metrics import mean_absolute_error
print("Mean Absolute Error : " + str(mean_absolute_error(predictions, test_y)))